In [1]:
import os
os.chdir("../")
# os.chdir("src/")
%pwd

'c:\\Users\\Nilesh\\Documents\\GitHub\\End-to-End-Diabetes-Detection'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [4]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
import os
from src.mlProject import logger
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

C:\Users\Nilesh\AppData\Local\Temp\ipykernel_389844\3012225129.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [9]:
from sklearn.preprocessing import StandardScaler

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        # Identify the indices of the independent variables
        independent_cols = data.columns[:-1]

        # Apply StandardScaler only to the independent variables
        scaler = StandardScaler()
        train[independent_cols] = scaler.fit_transform(train[independent_cols])
        test[independent_cols] = scaler.transform(test[independent_cols])

        # Reshape the target variable to 1-dimensional array
        y_train = train.iloc[:, -1].values.ravel()
        y_test = test.iloc[:, -1].values.ravel()

        # Save the transformed datasets
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)


In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-02-21 17:34:31,259: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-21 17:34:31,261: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-21 17:34:31,262: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-21 17:34:31,262: INFO: common: created directory at: artifacts]
[2024-02-21 17:34:31,262: INFO: common: created directory at: artifacts/data_transformation]
[2024-02-21 17:34:31,297: INFO: 4142836313: Split data into training and test sets]
[2024-02-21 17:34:31,298: INFO: 4142836313: (576, 8)]
[2024-02-21 17:34:31,298: INFO: 4142836313: (192, 8)]
(576, 8)
(192, 8)
